# Test-benchmark using fake data

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import warnings
warnings.filterwarnings("ignore")

# to remplace with pip install abench and uqmodels
path = '../'
sys.path.insert(1, path)
sys.path.insert(1, path+'src')
import abench as abench
import uqmodels as uqmodels
import abench.store as A_store
import matplotlib.pyplot as plt
import numpy as np
import pickle
# Check that current directory is at root level

# Data preprocessing and setup as Data_generator

In [12]:
#Data_processor for preprocessed data stored in a dict
from abench.benchmark import dataset_generator_from_array,splitter,TimeSeriesSplit,analyse_data_generator
from Benchmark_UQ import Encapsulated_model_UQ,TimeSeries_from_dict
from attack_function import ctx_seq_attack_UC_AL, ctx_seq_attack, test_attack

# Synthetic data
# Real data
Folder_data = path + 'data/gas_demand_data/'
Name_data = 'Dataset_real_data'
Real_dataset = TimeSeries_from_dict(Folder_data+Name_data)
Real_dataset.process()
X,y,context,train,test, X_split = Real_dataset.get_data()
X_shape,y_shape = X.shape[-1],y.shape[-1]
X_train, X_test, y_train, y_test, context_train,context_test = Real_dataset.split_train_test()
context[:,0] = context[:,2]*53 + context[:,3] - 13
X_split = X_split-3
X_split[X_split<0]=0
sk_split=splitter(X_split)

# Preliminary setup and subset partition recovery from data_generator
def compute_ctx_var_clustering(y,context,n_ctx,class_number):
    def in_list(idx,list_cluster):
        for ind_cluster,list_idx in enumerate(list_cluster):
            if(idx in list_idx):
                return(ind_cluster)
    std_values = [y[context[:,1]==i].std(axis=0).mean() for i in set(context[:,n_ctx])]
    ind_sort = np.argsort(std_values)

    ind_list=[]
    for i,n in enumerate(ind_sort):
        ind_list.append(n)

    class_area_cat = np.array_split(ind_sort,class_number)
    class_area_cat = [ind_sort[0:8],ind_sort[8:23],ind_sort[23:]]
    class_area_ctx = [in_list(i,class_area_cat)for i in context[:,1]]
    order_ctx = [ind_list.index(i) for i in context[:,1]]
    new_context = np.concatenate([context,np.array(order_ctx)[:,None],np.array(class_area_ctx)[:,None]],axis=1)
    return(new_context)
context = compute_ctx_var_clustering(y,context,1,3)

# Build data genarator
objective = None
dataset_generator = dataset_generator_from_array(X,y,context,objective,
                                                 sk_split=sk_split,
                                                 remove_from_train=None,
                                                 repetition=2,
                                                 attack_name='',
                                                 cv_list_name=['cv_0','cv_1'])

# Generation of training and inference variability injections
if(True):
    storing = path +'results/Benchmark_real_injection_learning'
    if not os.path.exists(storing):
        ratio_drop = 1 
        for ctx_deg in range(3): # 100 withdrawal training injection dataset experiments for the 3 subsets
            attack_str = '_deg'+str(ctx_deg)+'_seq'+str(int(ratio_drop*100))
            remove_from_train_seq_ctx0 = ctx_seq_attack_UC_AL(y,context,ctx_deg,ratio_drop,
                                                              len_past=12,len_futur=3,stick_seq=4)

            dataset_generator_deg = dataset_generator_from_array(X,y,context,objective,
                                                           sk_split=sk_split,
                                                           repetition=2,
                                                           remove_from_train=remove_from_train_seq_ctx0,
                                                           attack_name=attack_str,
                                                           cv_list_name=['cv_0','cv_1'])
            dataset_generator += dataset_generator_deg
            
        sratio_drop = 0.98
        for ctx_deg in range(3): # 98withdrawal training injection dataset experiments for the 3 subsets
            attack_str = '_deg'+str(ctx_deg)+'_seq'+str(int(ratio_drop*100))
            remove_from_train_seq_ctx0 = ctx_seq_attack_UC_AL(y,context,ctx_deg,ratio_drop,
                                                              len_past=12,len_futur=3,stick_seq=4)

            dataset_generator_deg = dataset_generator_from_array(X,y,context,objective,
                                                           sk_split=sk_split,
                                                           repetition=2,
                                                           remove_from_train=remove_from_train_seq_ctx0,
                                                           attack_name=attack_str,
                                                           cv_list_name=['cv_0','cv_1'])
            dataset_generator += dataset_generator_deg
        print('Generation of data-generator at '+storing)
        A_store.store_data_generator(storing,dataset_generator)
    else:
        dataset_generator=A_store.get_data_generator(storing)
        print(storing+' already exists')
if(False):
    from src.attack_function import injection_on_data_generator,attack_bis
    storing = path+'results/Benchmark_real_injection_inference'
    list_f = [20,28,17,16,23] # Number of Main features contributions according to SHAP
    type_f = ['Num','Num','Cat','Cat','Num','Num']
    type_attack = [('small',2),('small',5),('strong',2),('strong',5)]
    force = 0
    if not os.path.exists(storing):
        dataset_generator_with_attack = injection_on_data_generator(dataset_generator,type_attack,type_f,list_f,force)
        A_store.store_data_generator(storing,dataset_generator_with_attack)
    else:
        dataset_generator = A_store.get_data_generator(storing)
        print(storing+' already exists')

list_cv_name_all = [i[-1] for i in dataset_generator]
analyse_data_generator(dataset_generator)

../results/Benchmark_real_injection_learning already exists
% train for each sub sample for each set
cv_0  len 8473 %train/test/drop: [22.  43.1 17.1] [4.9 9.2 3.7] [0. 0. 0.]
cv_1  len 10297 %train/test/drop: [22.2 43.  17.1] [4.9 9.2 3.7] [0. 0. 0.]
cv_0_bis1  len 8473 %train/test/drop: [22.  43.1 17.1] [4.9 9.2 3.7] [0. 0. 0.]
cv_1_bis1  len 10297 %train/test/drop: [22.2 43.  17.1] [4.9 9.2 3.7] [0. 0. 0.]
cv_0_deg0_seq100  len 8473 %train/test/drop: [ 0.  43.1 17.1] [4.9 9.2 3.7] [22.  0.  0.]
cv_1_deg0_seq100  len 10297 %train/test/drop: [ 0.  43.  17.1] [4.9 9.2 3.7] [22.2  0.   0. ]
cv_0_deg0_seq100_bis1  len 8473 %train/test/drop: [ 0.  43.1 17.1] [4.9 9.2 3.7] [22.  0.  0.]
cv_1_deg0_seq100_bis1  len 10297 %train/test/drop: [ 0.  43.  17.1] [4.9 9.2 3.7] [22.2  0.   0. ]
cv_0_deg1_seq100  len 8473 %train/test/drop: [22.   0.  17.1] [4.9 9.2 3.7] [ 0.  43.1  0. ]
cv_1_deg1_seq100  len 10297 %train/test/drop: [22.2  0.  17.1] [4.9 9.2 3.7] [ 0. 43.  0.]
cv_0_deg1_seq100_bis1  le

# Modèle Formalisation

In [13]:
# Random forest UQ
from sklearn.ensemble import RandomForestRegressor
from uqmodels.common.basic_NN import build_BNN_loss,build_EDL_loss,build_MSE_loss,default_callbacks,mlp

step = 1000
callbacks = default_callbacks(min_delta=0.005,earlystop_patience=60,reducelr_patience=30,reducelr_factor=0.4,reduce_lr_min_lr=5e-06,verbose=0)

# MLP sans UQ
type_var = None
model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,200,150],
             'regularizer_W':(0.00005,0.00005),'name':'','dp':0.02,'type_var':type_var,
             'logvar_min':np.log(0.00001)}

training_param = {'epochs':[step,step,step],'b_s':[126,64,64],'l_r':[0.001,0.0005,0.0001],
                  'sample_w':None,'callbacks':callbacks}


MLP_parameters = {'rescale':False,
                  'model_initializer':mlp,
                  'model_parameters':model_param,
                  'training_parameters':training_param,
                  'type_var':type_var}

# RF_dUQ
RF_dUQ_parameters= {'estimator':RandomForestRegressor(ccp_alpha=5e-05, max_depth=18, max_features=0.8,
                                                      max_samples=1., min_impurity_decrease=5e-05,
                                                      min_samples_leaf=4,min_samples_split=6,
                                                      n_estimators=50),
                    'pretuned':False,'mode':'sigma','use_biais':True,'rescale':True}

# PNN_MCDP
type_var = 'MC_Dropout'
model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],"n_ech":6,
             'regularizer_W':(0.0004,0.0004),'name':'','dp':0.25,'type_var':type_var,
             'logvar_min':np.log(0.00001)}

training_param = {'epochs':[step,step,step],'b_s':[128,32,64],'l_r':[0.001,0.0005,0.0001],
                  'sample_w':None,'list_loss':[build_BNN_loss],
                  'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
                  'param_loss':[1.05],'callbacks':callbacks}

PNN_MCDP_parameters = {'rescale':False,
                     'model_initializer':mlp,
                     'model_parameters':model_param,
                     'training_parameters':training_param,
                     'type_var':type_var}
# PNN_DE
type_var = "Deep_ensemble"
model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
             'n_ech':5,'k_fold':8,"snapshot":False,'data_drop':0.05,'train_ratio':0.99,'ddof':1,
             'regularizer_W':(0.001,0.001),'name':'','dp':0.03,'type_var':type_var,
             'logvar_min':np.log(0.00001)}

training_param = {'epochs':[step,step],'b_s':[128,64],'l_r':[0.001,0.0005],'sample_w':None,
    'list_loss':[build_BNN_loss],'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
    'param_loss':[0.74],'callbacks':callbacks}


PNN_DE_parameters = {'rescale':False,'model_initializer':mlp,'model_parameters':model_param.copy(),
                     'training_parameters':training_param.copy(),'type_var':type_var}

#EDL
type_var='EDL'
model_param={'dim_in':X_shape,'dim_out':y_shape, 'layers_size':[200,300,200],
             'regularizer_W':(0.001,0.001),'name':'','dp':0.02,'type_var':type_var,
             'logvar_min':np.log(0.00005)}


training_param = {'epochs':[step,step,step],'b_s':[128,32,64],
                  'l_r':[0.001,0.0005,0.0001],'sample_w':None,'list_loss':[build_EDL_loss],
                  'metrics':[build_MSE_loss(4,True),build_BNN_loss(0.9,True,type_var)],
                  'callbacks':callbacks,'param_loss':[15e-2]}
              
EDL_parameters = {'rescale':False,'model_initializer':mlp,'model_parameters':model_param.copy(),
                  'training_parameters':training_param.copy(),'type_var':type_var}

# Ablation_studies parameters form mid-var with 100 withdrawal

In [14]:
def replace_in_dict(dict_,dict_update_values):
    for key in dict_update_values.keys():
        dict_[key]=dict_update_values[key]
        

#PNN_MCDP very-low 
dict_update = {'regularizer_W':(0.00001,0.00001),'dp':0.02,"n_ech":3}
replace_in_dict(model_param,dict_update)
replace_in_dict(training_param,{'param_loss':[0.13]})
PNN_MCDP_parameters_very_low = {'model_initializer':mlp, 'model_parameters':model_param.copy(),
                                        'training_parameters':training_param.copy(),'type_var':type_var}
#PNN_MCDP low 
dict_update = {'regularizer_W':(0.00001,0.00001),'dp':0.04,"n_ech":3}
replace_in_dict(model_param,dict_update)
replace_in_dict(training_param,{'param_loss':[0.15]})
PNN_MCDP_parameters_low = {'model_initializer':mlp, 'model_parameters':model_param.copy(),
                                        'training_parameters':training_param.copy(),'type_var':type_var}
#PNN_MCDP high-low 
dict_update = {'regularizer_W':(0.001,0.001),'dp':0.40,"n_ech":6}
replace_in_dict(model_param,dict_update)
replace_in_dict(training_param,{'param_loss':[1.6]})
PNN_MCDP_parameters_high = {'model_initializer':mlp, 'model_parameters':model_param.copy(),
                                        'training_parameters':training_param.copy(),'type_var':type_var}
#PNN_MCDP very-high
dict_update = {'regularizer_W':(0.001,0.001),'dp':0.50,"n_ech":6}
replace_in_dict(model_param,dict_update)
replace_in_dict(training_param,{'param_loss':[1.8]})
PNN_MCDP_parameters_very_high = {'model_initializer':mlp, 'model_parameters':model_param.copy(),
                                        'training_parameters':training_param.copy(),'type_var':type_var}

#PNN_DE very_low
dict_update = {'n_ech':2,'k_fold':None,"snapshot":True,'data_drop':0,'train_ratio':0.95,
               'regularizer_W':(0.00005,0.00005)}                                                                                                         
replace_in_dict(model_param,dict_update)
replace_in_dict(training_param,{'param_loss':[0.35]})
PNN_DE_parameters_very_low = {'model_initializer':mlp,'model_parameters':model_param.copy(),
                         'training_parameters':training_param.copy(),'type_var':type_var}

#PNN_DE low 
dict_update = {'n_ech':2,'k_fold':None,"snapshot":True,'data_drop':0,'train_ratio':0.95,
               'regularizer_W':(0.0004,0.0004)}                                                                                                         
replace_in_dict(model_param,dict_update)
replace_in_dict(training_param,{'param_loss':[0.55]})
PNN_DE_parameters_low = {'model_initializer':mlp,'model_parameters':model_param.copy(),
                         'training_parameters':training_param.copy(),'type_var':type_var}

#PNN_DE high
dict_update = {'n_ech':5,'k_fold':5,"snapshot":False,'data_drop':0.2,'train_ratio':0.99,
               'regularizer_W':(0.003,0.003)}
replace_in_dict(model_param,dict_update)
replace_in_dict(training_param,{'param_loss':[0.7]})
PNN_DE_parameters_high = {'model_initializer':mlp, 'model_parameters':model_param.copy(),
                                        'training_parameters':training_param.copy(),'type_var':type_var}

#PNN_DE very-high
dict_update = {'n_ech':5,'k_fold':5,"snapshot":False,'data_drop':0.30,'train_ratio':0.99,
               'regularizer_W':(0.005,0.005)}
replace_in_dict(model_param,dict_update)
replace_in_dict(training_param,{'param_loss':[0.6]})
PNN_DE_parameters_very_high = {'model_initializer':mlp, 'model_parameters':model_param.copy(),
                                        'training_parameters':training_param.copy(),'type_var':type_var}

#PNN_DE RF_very_low
RF_dUQ_parameters_very_low = {'estimator':RandomForestRegressor(ccp_alpha=1e-07, max_depth=25, max_features=0.8,
                                                      max_samples=1., min_impurity_decrease=1e-07,
                                                      min_samples_leaf=2, min_samples_split=2,
                                                      n_estimators=12),
                          'pretuned':False,'mode':'sigma','use_biais':True,'rescale':True}

#RF_dUQ_low
RF_dUQ_parameters_low = {'estimator':RandomForestRegressor(ccp_alpha=9e-06, max_depth=20, max_features=0.8,
                                                      max_samples=1., min_impurity_decrease=9e-06,
                                                      min_samples_leaf=2, min_samples_split=2,
                                                      n_estimators=30),
                     'pretuned':False,'mode':'sigma','use_biais':True,'rescale':True}

#RF_dUQ_high
RF_dUQ_parameters_high = {'estimator':RandomForestRegressor(ccp_alpha=1e-05, max_depth=15, max_features=0.8,
                                                        max_samples=1., min_impurity_decrease=1e-05,
                                                        min_samples_leaf=10,min_samples_split=10,
                                                        n_estimators=50),
                      'pretuned':False,'mode':'sigma','use_biais':True,'rescale':True}
#RF_dUQ RF_very_high
RF_dUQ_parameters_very_high = {'estimator':RandomForestRegressor(ccp_alpha=1e-05, max_depth=12,max_features=0.8,
                                                                 max_samples=1., 
                                                             min_impurity_decrease=1e-05,
                                                             min_samples_leaf=20,min_samples_split=20,
                                                             n_estimators=50),
                           'pretuned':False,'mode':'sigma','use_biais':True,'rescale':True}

# Benchmark formalisation

In [15]:
random = np.random.randint(10000000)
from uqmodels.common.neural_network_UQ import NN_var
from uqmodels.common.random_forest_UQ import PredictorRF_UQ_distangle

# Transmision des modèles / Initialisateurs de modèles
dict_Predictor={}
dict_Predictor['MLP'] = {'subpart':NN_var,'parameters':MLP_parameters}
dict_Predictor['RF_dUQ'] = {'subpart':PredictorRF_UQ_distangle,'parameters':RF_dUQ_parameters}
dict_Predictor['EDL'] = {'subpart':NN_var,'parameters':EDL_parameters}      
dict_Predictor['PNN_DE'] = {'subpart':NN_var,'parameters':PNN_DE_parameters}
dict_Predictor['PNN_MCDP'] = {'subpart':NN_var,'parameters':PNN_MCDP_parameters}

exp_design = []
# Spécification du plan d'expérience
exp_design.append([{'name':'MLP_bis','model':'MLP'},
                   {'name':'RF_dUQ_bis','model':'RF_dUQ'},
                   {'name':'PNN_MCDP_bis','model':'PNN_MCDP'},
                   {'name':'PNN_DE_bis','model':'PNN_DE'},
                   {'name':'EDL_bis','model':'EDL'}])

# Model wrapper :
from src.Benchmark_UQ import Encapsulated_model_UQ
# Configuration transmis à abench coté modèle
dict_exp={'encapsulated_model': Encapsulated_model_UQ,
          'tuning_scheme' : {'model':None},
          'model' : dict_Predictor,
          'exp_design':exp_design}



In [ ]:
from src.Benchmark_UQ import rmse,average_coverage,sharpness,Gaussian_NLL
from abench.benchmark import Generic_metric

# Metrics definition to encapsulate in Generic_metric
storing = path +'results/Benchmark_real_injection_learning'
# or storing = path +'results/Benchmark_real_injection_inference'
cv_list =  None

from abench.benchmark import Generic_metric

list_metrics=[Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=None,reduce=True),
              Generic_metric(average_coverage,"Cov_ALL", mask=None,list_ctx_constraint=None,reduce=True,type_var="all"),
              Generic_metric(Gaussian_NLL,"NLL", mask=None,list_ctx_constraint=None,reduce=True,type_var="all"),
              Generic_metric(sharpness,"Aw", mask=None,list_ctx_constraint=None,reduce=True,type_var="all")]

obj_param = {'alpha':0.05}
tuning_kwarg = {}

#Start from empty dict

from abench.benchmark import benchmark
dataset_generator = None
# Run benchmark'Benchmark_UC_AL_0'
storing = benchmark(storing,dataset_generator,dict_exp,obj_param,
                    list_metrics,tuning_kwarg=tuning_kwarg,verbose=0,cv_list=cv_list)

In [5]:
from abench.benchmark import evaluate, Generic_metric
from src.Benchmark_UQ import rmse,average_coverage,sharpness,Gaussian_NLL
storing = path+'results/Benchmark_real_injection_learning'
cv_list = ['cv_0','cv_1','cv_0_bis1','cv_1_bis1']
list_name = ['MLP','RF_dUQ','PNN_MCDP','PNN_DE','EDL']
list_ctx_constraint=None #[(-2,None,0.5)]
list_metrics = [Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True),
                Generic_metric(Gaussian_NLL,"NLL", mask=None,list_ctx_constraint=None,reduce=True),
                Generic_metric(average_coverage,"Cov_all", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True),
                Generic_metric(sharpness,"Aw", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True,type_var='all'),
               ]
dict_perf = evaluate(storing, list_name, list_metrics,verbose=1,cv_list=cv_list)

cv_0 len_train 6965
cv_1 len_train 8473
cv_0_bis1 len_train 6965
cv_1_bis1 len_train 8473
MLP  |time_fit : 216.82 time_pred : 2.22
Rmse_all Train 0.151 ± 0.003 | TEST 0.216 ± 0.019
NLL Train 1.271 ± 0.0 | TEST 1.277 ± 0.002
Cov_all Train 1.0 ± 0.0 | TEST 1.0 ± 0.0
Aw Train 5.644 ± 0.0 | TEST 5.644 ± 0.0

RF_dUQ  |time_fit : 3.56 time_pred : 4.18
Rmse_all Train 0.143 ± 0.003 | TEST 0.236 ± 0.015
NLL Train -0.798 ± 0.015 | TEST -0.506 ± 0.053
Cov_all Train 0.991 ± 0.001 | TEST 0.951 ± 0.01
Aw Train 0.725 ± 0.009 | TEST 0.831 ± 0.01

PNN_MCDP  |time_fit : 584.74 time_pred : 24.79
Rmse_all Train 0.17 ± 0.004 | TEST 0.226 ± 0.017
NLL Train -0.804 ± 0.017 | TEST -0.548 ± 0.075
Cov_all Train 0.986 ± 0.001 | TEST 0.949 ± 0.013
Aw Train 0.766 ± 0.012 | TEST 0.813 ± 0.007

PNN_DE  |time_fit : 2036.57 time_pred : 5.98
Rmse_all Train 0.153 ± 0.004 | TEST 0.211 ± 0.019
NLL Train -0.848 ± 0.017 | TEST -0.581 ± 0.087
Cov_all Train 0.99 ± 0.001 | TEST 0.952 ± 0.013
Aw Train 0.702 ± 0.021 | TEST 0.73 ±